In [1]:
import pickle
import os
import numpy as np
import cv2

In [9]:
label_dir = "../data/coco/voc07_val.pkl"
result_dir = "../work_dirs/cascade_rcnn_dconv_c3-c5_r101_fpn_1x/result_eval.pkl"
img_dir = "../data/coco/train2017/"

In [21]:
plot_error_bbox(label_dir, result_dir)
# read_pkl(label_dir)

(44, 44)
GT: 44, Predict:44
--------
(114, 114)
GT: 114, Predict:114
--------
(114, 114)
GT: 114, Predict:114
--------
(181, 181)
180 not match
GT: 181, Predict:181
--------
(124, 124)
GT: 124, Predict:124
--------
(160, 159)
159 not match
GT: 159, Predict:160
--------
(115, 115)
GT: 115, Predict:115
--------
(80, 80)
GT: 80, Predict:80
--------
(264, 266)
214 not match
262 not match
GT: 266, Predict:264
--------
(62, 62)
GT: 62, Predict:62
--------
(62, 62)
GT: 62, Predict:62
--------
(93, 93)
GT: 93, Predict:93
--------
(224, 224)
219 not match
GT: 224, Predict:224
--------
(115, 115)
GT: 115, Predict:115
--------
(107, 107)
GT: 107, Predict:107
--------
(145, 145)
GT: 145, Predict:145
--------
(62, 62)
GT: 62, Predict:62
--------
(84, 84)
GT: 84, Predict:84
--------
(96, 96)
GT: 96, Predict:96
--------
(247, 259)
186 not match
197 not match
225 not match
231 not match
243 not match
GT: 259, Predict:247
--------
(115, 115)
GT: 115, Predict:115
--------
(119, 119)
GT: 119, Predict:119

In [20]:
def plot_error_bbox(label_path, predict_path):
    label_data = read_pkl(label_path)
    predict_data = read_pkl(predict_path)
    iou_thr = 0.7
    tpfp = []
    
    cls_gt_ignore = [] 
    gt_ignore = []
    for i, each_data in enumerate(label_data):
        bboxes = each_data["ann"]["bboxes"]
        ignore = np.concatenate([
            np.zeros(bboxes.shape[0], dtype=np.bool),
            np.ones(0, dtype=np.bool)
        ])
        # gt_ignore.append(ignore)
        cls_gt_ignore.append(ignore)

             
    for i, each_data in enumerate(label_data):
        file_name = os.path.join(img_dir, each_data["filename"])
        gd_bboxes = each_data["ann"]["bboxes"]
        predict_bboxes = predict_data[i][0]
        
        tp, fp = tpfp_func(predict_bboxes, gd_bboxes, cls_gt_ignore[i], iou_thr, None) # j 表示图像索引
        tpfp.append([tp, fp]) 
    
        
        tp = np.array(1 - tp[0], dtype=np.bool)   #  not tp :误检
        fp = np.array(fp[0], dtype=np.bool)  # fp: 漏检
        
        # 画漏检的样本
        img = cv2.imread(file_name)
        print("GT: {}, Predict:{}".format(len(gd_bboxes), len(predict_bboxes)))
        # omite_gt_bboxes = gd_bboxes[fp]
        if len(predict_bboxes) < len(gd_bboxes):
            omite_detected_bboxes = predict_bboxes[fp]
            for i in range(len(omite_detected_bboxes)):
                if len(omite_detected_bboxes) == len(gd_bboxes):
                    cv2.rectangle(img, (gd_bboxes[i][0], gd_bboxes[i][1]), (gd_bboxes[i][2], gd_bboxes[i][3]), (0, 0, 255), 2)
                cv2.rectangle(img, (omite_detected_bboxes[i][0], omite_detected_bboxes[i][1]), (omite_detected_bboxes[i][2], omite_detected_bboxes[i][3]), (0, 0, 255), 2)
                cv2.imwrite(os.path.join("../work_dirs/omite_detected_bboxes/", os.path.basename(file_name)), img)
            
        # 画误检样本
        img = cv2.imread(file_name)
        # error_gt_bboxes = gd_bboxes[tp]
        if len(predict_bboxes) > len(gd_bboxes):
            error_detected_bboxes = predict_bboxes[tp]
            for i in range(len(error_detected_bboxes)):
                if len(error_detected_bboxes) == len(gd_bboxes):
                    cv2.rectangle(img, (gd_bboxes[i][0], gd_bboxes[i][1]), (gd_bboxes[i][2], gd_bboxes[i][3]), (0, 255, 0), 2)
                cv2.rectangle(img, (error_detected_bboxes[i][0], error_detected_bboxes[i][1]), (error_detected_bboxes[i][2], error_detected_bboxes[i][3]), (0, 0, 255), 2)
                cv2.imwrite(os.path.join("../work_dirs/error_detected_bboxes/", os.path.basename(file_name)), img)
        
        
        # print(tp, fp)
        print("--------")
  

In [4]:
def read_pkl(pkl_path):
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
        # print(data)
        '''
        print("{} data label".format(len(data)))
        for idx, each in enumerate(data):
            print("image: {} have {} bboxes".format(idx, len(each[0])))
        '''
        return data

In [5]:
def bbox_iou(bbox_a, bbox_b):
    """
    以bbox1 = np.array([[50, 50, 100, 100],[80, 80, 125, 125], [90, 90, 125, 
    125]]).reshape(3, 4)
         bbox2 = np.array([[60, 60, 120, 120], [40, 40, 60, 60]]).reshape(2, 4)为例
    """
    if bbox_a.shape[1] != 4 or bbox_b.shape[1] != 4:
        raise IndexError
 
    tl = np.maximum(bbox_a[:, None, :2], bbox_b[:, :2]) #这里是[3, 1, 2]和[2,2]运算
    #自动扩充到[3, 2, 2]和[3, 2, 2]运算 下面也一样
    br = np.minimum(bbox_a[:, None, 2:], bbox_b[:, 2:])
 
    area_i = np.prod(br - tl, axis=2) * (tl < br).all(axis=2)#[3, 2]
    area_a = np.prod(bbox_a[:, 2:] - bbox_a[:, :2], axis=1) #[3, ]
    area_b = np.prod(bbox_b[:, 2:] - bbox_b[:, :2], axis=1) #[2, ]
    return area_i / (area_a[:, None] + area_b - area_i)
    #      [3, 2] / ([3, 1]           + [2, ] - [3, 2])


In [16]:
def tpfp_func(det_bboxes, gt_bboxes, gt_ignore, iou_thr, area_ranges=None):
    """Check if detected bboxes are true positive or false positive.

    Args:
        det_bbox (ndarray): the detected bbox
        gt_bboxes (ndarray): ground truth bboxes of this image
        gt_ignore (ndarray): indicate if gts are ignored for evaluation or not
        iou_thr (float): the iou thresholds

    Returns:
        tuple: (tp, fp), two arrays whose elements are 0 and 1
    """
    num_dets = det_bboxes.shape[0]
    num_gts = gt_bboxes.shape[0]
    if area_ranges is None:
        area_ranges = [(None, None)]
    num_scales = len(area_ranges)
    # tp and fp are of shape (num_scales, num_gts), each row is tp or fp of
    # a certain scale
    tp = np.zeros((num_scales, num_dets), dtype=np.float32)
    fp = np.zeros((num_scales, num_dets), dtype=np.float32)
    # if there is no gt bboxes in this image, then all det bboxes
    # within area range are false positives
    if gt_bboxes.shape[0] == 0:
        if area_ranges == [(None, None)]:
            fp[...] = 1
        else:
            det_areas = (det_bboxes[:, 2] - det_bboxes[:, 0] + 1) * (
                det_bboxes[:, 3] - det_bboxes[:, 1] + 1)
            for i, (min_area, max_area) in enumerate(area_ranges):
                fp[i, (det_areas >= min_area) & (det_areas < max_area)] = 1
        return tp, fp
    ious = bbox_overlaps(det_bboxes, gt_bboxes)
    print(np.array(ious).shape)
    ious_max = ious.max(axis=1) # 对每个Grond Truth找其匹配的最大IOU
    ious_argmax = ious.argmax(axis=1) # 对每个Grond Truth找其匹配的最大IOU的索引
    sort_inds = np.argsort(-det_bboxes[:, -1])  # 降序排列检测出的bbox
    for k, (min_area, max_area) in enumerate(area_ranges):
        gt_covered = np.zeros(num_gts, dtype=bool)
        # if no area range is specified, gt_area_ignore is all False
        if min_area is None:
            gt_area_ignore = np.zeros_like(gt_ignore, dtype=bool)
        else:
            gt_areas = (gt_bboxes[:, 2] - gt_bboxes[:, 0] + 1) * (
                gt_bboxes[:, 3] - gt_bboxes[:, 1] + 1)
            gt_area_ignore = (gt_areas < min_area) | (gt_areas >= max_area)
        for i in sort_inds:  # 对每一个bbox
            if ious_max[i] >= iou_thr:
                matched_gt = ious_argmax[i]
                if not (gt_ignore[matched_gt] or gt_area_ignore[matched_gt]):
                    if not gt_covered[matched_gt]:
                        gt_covered[matched_gt] = True
                        tp[k, i] = 1
                    else:
                        fp[k, i] = 1
                        print(i, "not match")
                # otherwise ignore this detected bbox, tp = 0, fp = 0
            elif min_area is None:
                fp[k, i] = 1
                print(i, "not match")
            else:
                bbox = det_bboxes[i, :4]
                area = (bbox[2] - bbox[0] + 1) * (bbox[3] - bbox[1] + 1)
                if area >= min_area and area < max_area:
                    fp[k, i] = 1
                    print(i, "not match")
    return tp, fp


In [7]:
def bbox_overlaps(bboxes1, bboxes2, mode='iou'):
    """Calculate the ious between each bbox of bboxes1 and bboxes2.

    Args:
        bboxes1(ndarray): shape (n, 4)
        bboxes2(ndarray): shape (k, 4)
        mode(str): iou (intersection over union) or iof (intersection
            over foreground)

    Returns:
        ious(ndarray): shape (n, k)
    """

    assert mode in ['iou', 'iof']

    bboxes1 = bboxes1.astype(np.float32)
    bboxes2 = bboxes2.astype(np.float32)
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    ious = np.zeros((rows, cols), dtype=np.float32)
    if rows * cols == 0:
        return ious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        ious = np.zeros((cols, rows), dtype=np.float32)
        exchange = True
    area1 = (bboxes1[:, 2] - bboxes1[:, 0] + 1) * (
        bboxes1[:, 3] - bboxes1[:, 1] + 1)
    area2 = (bboxes2[:, 2] - bboxes2[:, 0] + 1) * (
        bboxes2[:, 3] - bboxes2[:, 1] + 1)
    for i in range(bboxes1.shape[0]):
        x_start = np.maximum(bboxes1[i, 0], bboxes2[:, 0])
        y_start = np.maximum(bboxes1[i, 1], bboxes2[:, 1])
        x_end = np.minimum(bboxes1[i, 2], bboxes2[:, 2])
        y_end = np.minimum(bboxes1[i, 3], bboxes2[:, 3])
        overlap = np.maximum(x_end - x_start + 1, 0) * np.maximum(
            y_end - y_start + 1, 0)
        if mode == 'iou':
            union = area1[i] + area2 - overlap
        else:
            union = area1[i] if not exchange else area2
        ious[i, :] = overlap / union
    if exchange:
        ious = ious.T
    return ious
